In [1]:
import spacy
from spacy_llm.util import assemble
import spacy
import pandas as pd
from tqdm import tqdm
from youbot.store import Store


nlp = spacy.load("en_core_web_md")
nlp = assemble("config.cfg")

/Users/tombedor/development/youbot/.venv/lib/python3.11/site-packages/spacy_llm/models/hf/base.py:142: UserWarning: Couldn't find a CUDA GPU, so the setting 'device_map:auto' will be used, which may result in the LLM being loaded (partly) on the CPU or even the hard disk, which may be slow. Install cuda to be able to load and run the LLM on the GPU instead.
  warnings.warn(
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


ValueError: Could not load model databricks/dolly-v2-3b with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.gpt_neox.modeling_gpt_neox.GPTNeoXForCausalLM'>). See the original errors:

while loading with AutoModelForCausalLM, an error is thrown:
Traceback (most recent call last):
  File "/Users/tombedor/development/youbot/.venv/lib/python3.11/site-packages/transformers/pipelines/base.py", line 283, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/tombedor/development/youbot/.venv/lib/python3.11/site-packages/transformers/models/auto/auto_factory.py", line 563, in from_pretrained
    return model_class.from_pretrained(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/tombedor/development/youbot/.venv/lib/python3.11/site-packages/transformers/modeling_utils.py", line 3677, in from_pretrained
 

In [ ]:
MESSAGES_COUNT = 10
docs = Store().get_archival_messages()  # [:MESSAGES_COUNT]


entities_df = pd.DataFrame(columns=["name", "label", "fact"])
relations_df = pd.DataFrame(columns=["dep_name", "dep_label", "dest_name", "dest_label", "rel", "fact"])


entities_rows = []
reltaions_rows = []
for doc in tqdm(docs):
    enriched_doc = nlp(doc)
    ents = enriched_doc.ents
    for ent in ents:
        entities_rows.append({"name": ent.text, "label": ent.label_, "fact": doc})  # type: ignore
    for rel in enriched_doc._.rel:
        dep_name = ents[rel.dep].text
        dep_label = ents[rel.dep].label_
        dest_name = ents[rel.dest].text
        dest_label = ents[rel.dest].label_

        # ignore self relations
        if (dep_name, dep_label) == (dest_name, dest_label):
            continue

        # ignore dates
        if dep_label == "DATE" or dest_label == "DATE":
            continue

        reltaions_rows.append(
            {
                "dep_name": dep_name,
                "dep_label": dep_label,
                "dest_name": dest_name,
                "dest_label": dest_label,
                "rel": rel.relation,
                "fact": doc,
            }
        )


entities_df = pd.DataFrame(entities_rows)
relations_df = pd.DataFrame(reltaions_rows)

In [1]:
# group facts from entities
grouped_entities_df = entities_df.groupby(["name", "label"]).agg({"fact": lambda x: list(x)}).reset_index()
grouped_relations_df = (
    relations_df.groupby(["dep_name", "dep_label", "dest_name", "dest_label", "rel"]).agg({"fact": lambda x: list(x)}).reset_index()
)


# in some passes, a subset of the labels should be discarded

# perhaps, characterize the convo by work, personal, etc and tailor retrieval accordingly



NameError: name 'entities_df' is not defined

In [ ]:
# clashing entity types
labels_grouped = grouped_entities_df.groupby(['name']).agg({'label': lambda x: list(x)}).reset_index()
clashing_entity_labels = labels_grouped[labels_grouped['label'].apply(lambda x: len(x) > 1)]


rels_grouped = grouped_relations_df.groupby(['dep_name', 'dest_name']).agg({'rel': lambda x: list(x)}).reset_index()
clashing_rels = rels_grouped[rels_grouped['rel'].apply(lambda x: len(x) > 1)]



In [ ]:
for i, row in clashing_entity_labels.iterrows():
    print(row['name'], row['label'])
    for label in row['label']:
        print(entities_df[(entities_df['name'] == row['name']) & (entities_df['label'] == label)]['fact'].values[0])